## Plagiarism Detection

### Main and related tasks in plagiarism detection

* **Plagiarism detection:** Given a document, identify all  plagiarized sources and boundaries of re-used passages.
   - similar to deduplication
* **Author identification:** Given a document, identify its author.
* **Author profiling:** Given a document, extract information about the author (e.g. gender, age).

### External vs. Intrinsic plagiarism detection

#### External plagiarism detection

Given a set of suspicious documents and a set of source documents the
task is to find all text passages in the suspicious documents which have
been plagiarized and the corresponding text passages in the source
documents.

#### Intrinsic plagiarism detection

Given a set of suspicious documents the task is to identify all plagiarized
text passages, e.g., by detecting writing style breaches. The comparison of
a suspicious document with other documents is not allowed in this task.

# Task: Select a detection algorithm and implement it in Python

- Input: File in a 3-column vertical format (word, lemma, tag)
- Output: One plagiarism per line: id TAB detected source id TAB real source id. Evaluation line: precision, recall F1 measure.


In [1]:
!wget https://nlp.fi.muni.cz/trac/research/raw-attachment/wiki/en/AdvancedNlpCourse/LanguageResourcesFromWeb/training_data.vert

--2021-10-27 18:05:12--  https://nlp.fi.muni.cz/trac/research/raw-attachment/wiki/en/AdvancedNlpCourse/LanguageResourcesFromWeb/training_data.vert
Resolving nlp.fi.muni.cz (nlp.fi.muni.cz)... 147.251.51.11
Connecting to nlp.fi.muni.cz (nlp.fi.muni.cz)|147.251.51.11|:443... connected.
HTTP request sent, awaiting response... 200 Ok
Length: 503730 (492K) [application/octet-stream]
Saving to: ‘training_data.vert’

training_data.vert  100%[===================>] 491.92K   672KB/s    in 0.7s    

2021-10-27 18:05:14 (672 KB/s) - ‘training_data.vert’ saved [503730/503730]



In [68]:
!head -n50 training_data.vert

<doc author="Josef Plch" id="101" class="original" source="101">
<s>
Reveň	reveň	k1gFnSc1
kadeřavá	kadeřavý	k2eAgFnSc1d1
Reveň	reveň	k1gFnSc1
kadeřavá	kadeřavý	k2eAgFnSc1d1
(	(	kIx(
<g/>
Rheum	Rheum	k1gInSc1
rhabarbarum	rhabarbarum	k1gInSc1
<g/>
)	)	kIx)
je	být	k5eAaImIp3nS
rostlina	rostlina	k1gFnSc1
náležící	náležící	k2eAgFnSc1d1
do	do	k7c2
rodu	rod	k1gInSc2
reveň	reveň	k1gFnSc1
<g/>
,	,	kIx,
čeledi	čeleď	k1gFnSc3
rdesnovité	rdesnovitý	k2eAgFnSc2d1
<g/>
.	.	kIx.
</s>
<s desamb="1">
Řapíky	řapík	k1gInPc1
listů	list	k1gInPc2
se	sebe	k3xPyFc4
užívají	užívat	k5eAaImIp3nP
jako	jako	k8xC,k8xS
zelenina	zelenina	k1gFnSc1
<g/>
,	,	kIx,
známá	známý	k2eAgFnSc1d1
pod	pod	k7c7
jménem	jméno	k1gNnSc7
rebarbora	rebarbora	k1gFnSc1
(	(	kIx(
<g/>
z	z	k7c2
latinského	latinský	k2eAgInSc2d1
názvu	název	k1gInSc2
<g/>
)	)	kIx)
<g/>
.	.	kIx.
</s>
<s desamb="1">
Synonyma	synonymum	k1gNnPc1


In [101]:
import sys, codecs, re

def parse_input(input):
  """
  Parse input vert file into dictionary. On top level, documents are grouped by authors. 
  Each document is represented by dictionary with metadata
    - author, 
    - unique id, 
    - class (original or suspicious), 
    - source_id (The same as unique id for originals. Referencing original unique id for suspicious documents.),
    - wordlist (set of words with their counts)
    - lemmalist (set of lemmas with their counts)
  """

  header_re = re.compile('<doc author="([^"]+)" id="(\d+)" class="(plagiarism|original)" source="(\d+)"')

  # reading all docurment into the memory - okey for small amout
  doc_sets = {} # sets of documents, each from one author
  doc = {}
  word_set = {}
  lemma_set = {}
  N = 0
  with open(input, "r") as handle:
    for line in handle:
        if line.startswith('<doc'):

            # structure for info about document
            author, id_, class_, source_id = header_re.match(line).groups()
            doc = {
                'author': author,
                'id': id_,
                'class': class_,
                'source_id': source_id,
                'wordlist': {},
                'lemmalist': {}
            }
        elif line.startswith('</doc'):

            # adding document to author's set - to original of suspisious documents
            if not doc['author'] in doc_sets:
                doc_sets[doc['author']] = {'original': [], 'suspicious': []}
            if doc['class'] == 'original':
                doc_sets[doc['author']]['original'].append(doc)
            else:
                doc_sets[doc['author']]['suspicious'].append(doc)

            N += 1
        elif not line.startswith('<'):

            # adding info about content of document
            word, lemma, tag = line.rstrip().split('\t')[:3]
            doc['wordlist'][word] = doc['wordlist'].get(word, 0) + 1
            doc['lemmalist'][lemma] = doc['lemmalist'].get(lemma, 0) + 1

            word_set[word] = word_set.get(word, 0) + 1
            lemma_set[lemma] = lemma_set.get(lemma, 0) + 1

    return doc_sets, lemma_set, N

In [53]:
DOC_SIMILARITY_THRESHOLD = 0.5
from scipy import spatial

def cosine_similarity_wordlist(doc1, doc2, *kwargs):
    """
    Converting documents into vectors and computing their cosine distance.
    Each item of a vector represents one word, value of that item represents
    relative counts of a word. 
    Result is a number betwwen 0 and 1 representing similarity of documents 
    (1 meand identity).
    """
    vector1, vector2 = [], []
    all_words = list(doc1['wordlist'].keys() | doc2['wordlist'].keys())
    doc1_len = float(sum(doc1['wordlist'].values()))
    doc2_len = float(sum(doc2['wordlist'].values()))
    for word in all_words:
        vector1.append(doc1['wordlist'].get(word, 0) / doc1_len)
        vector2.append(doc2['wordlist'].get(word, 0) / doc2_len)
    cosine_similarity = 1.0 - spatial.distance.cosine(vector1, vector2)
    return cosine_similarity

In [54]:
def cosine_similarity_lemmalist(doc1, doc2, *kwargs):
    """
    Converting documents into vectors and computing their cosine distance.
    Each item of a vector represents one word, value of that item represents
    relative counts of a word. 
    Result is a number betwwen 0 and 1 representing similarity of documents 
    (1 meand identity).
    """
    vector1, vector2 = [], []
    all_words = list(doc1['lemmalist'].keys() | doc2['lemmalist'].keys())
    doc1_len = float(sum(doc1['lemmalist'].values()))
    doc2_len = float(sum(doc2['lemmalist'].values()))
    for word in all_words:
        vector1.append(doc1['lemmalist'].get(word, 0) / doc1_len)
        vector2.append(doc2['lemmalist'].get(word, 0) / doc2_len)
    cosine_similarity = 1.0 - spatial.distance.cosine(vector1, vector2)
    return cosine_similarity

In [97]:
import numpy as np

def tf(word, doc):
    N = float(sum(doc['lemmalist'].values()))
    return doc['lemmalist'][word]/N

def idf(word, lemma_set, N):
    try:
        word_occurance = lemma_set[word] + 1
    except:
        word_occurance = 1
    return np.log(N/word_occurance)

def tf_idf(doc, lemma_set, N):
    tf_idf_vec = np.zeros((len(lemma_set),))
    for word in doc['lemmalist'].keys():
        tf_ = tf(word, doc)
        idf_ = idf(word, lemma_set, N)
          
        value = tf_*idf_
        tf_idf_vec[lemma_set[word]] = value 
    return tf_idf_vec

def tfidf_lemma(doc1, doc2, lemma_set, N): 

    vector1 = tf_idf(doc1, lemma_set, N)
    vector2 = tf_idf(doc2, lemma_set, N)
    cosine_similarity = 1.0 - spatial.distance.cosine(vector1, vector2)
    return cosine_similarity
  

In [103]:
for author, doc_set in doc_sets.items():
      for doc in doc_set['suspicious']:
        tf_idf(doc, lemma_set, N)

In [56]:
def evaluate(doc_sets, lemma_set, N, metric):
  #Srovname wordlisty podezrelych dokumentu s originaly ze stejne sady dokumentu.
  #Zaroven vyhodnocujeme uspesnost.
  stats = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}
  for author, doc_set in doc_sets.items():
      print('Doc set by %s\n' % author)
      set_stats = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}
      for doc in doc_set['suspicious']:
          #srovnani se vsemi originaly
          most_similar_doc_id = doc['id'] #vychozi stav je dokument je nejpodobnejsi sam sobe
          highest_similarity_score = 0.0
          plagiarism = False
          for orig_doc in doc_set['original']:
              similarity_score = metric(doc, orig_doc, lemma_set, N)
              if similarity_score >= DOC_SIMILARITY_THRESHOLD \
                      and similarity_score > highest_similarity_score:
                  most_similar_doc_id = orig_doc['id']
                  highest_similarity_score = similarity_score
                  plagiarism = True
          print('%s\t%s\t%s\n' % (doc['id'], most_similar_doc_id, doc['source_id']))
          #vyhodnoceni
          if most_similar_doc_id == doc['source_id']:
              if doc['class'] == 'plagiarism':
                  set_stats['tp'] += 1
              else:
                  set_stats['tn'] += 1
          else:
              if doc['class'] == 'plagiarism':
                  set_stats['fp'] += 1
              else:
                  set_stats['fn'] += 1
      #vyhodnoceni
      try:
          precision = set_stats['tp'] / float(set_stats['tp'] + set_stats['fp'])
      except ZeroDivisionError:
          precision = 0.0
      try:
          recall = set_stats['tp'] / float(set_stats['tp'] + set_stats['fn'])
      except ZeroDivisionError:
          recall = 0.0
      try:
          f1_measure = 2 * precision * recall / (precision + recall)
      except ZeroDivisionError:
          f1_measure = 0.0
      print('Set precision: %.2f, recall: %.2f, F1: %.2f\n\n' %
          (precision, recall, f1_measure))
      stats['tp'] += set_stats['tp']
      stats['fp'] += set_stats['fp']
      stats['tn'] += set_stats['tn']
      stats['fn'] += set_stats['fn']
  try:
      precision = stats['tp'] / float(stats['tp'] + stats['fp'])
  except ZeroDivisionError:
      precision = 0.0
  try:
      recall = stats['tp'] / float(stats['tp'] + stats['fn'])
  except ZeroDivisionError:
      recall = 0.0
  try:
      f1_measure = 2 * precision * recall / (precision + recall)
  except ZeroDivisionError:
      f1_measure = 0.0
  print('Overall precision: %.2f, recall: %.2f, F1: %.2f\n' %
      (precision, recall, f1_measure))

In [102]:
doc_sets, lemma_set, N = parse_input('training_data.vert')
evaluate(doc_sets, lemma_set, N, cosine_similarity_wordlist)

Doc set by Josef Plch

106	101	101

107	103	103

108	104	104

109	104	104

110	105	105

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Vojtěch Škvařil

202	201	201

204	203	203

206	205	205

208	207	207

210	209	209

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Nikol Volková

306	301	301

307	302	302

308	303	303

309	304	304

310	305	305

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Lucie Kaplanová

402	401	401

404	403	403

406	405	405

408	407	407

410	409	409

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Matej Gallo

506	501	501

507	501	501

508	502	502

509	502	503

510	502	504

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Lukas Banic

606	605	601

607	602	602

609	605	604

610	605	605

613	605	611

Set precision: 0.40, recall: 1.00, F1: 0.57


Doc set by 422765

702	701	701

704	703	703

706	705	705

708	707	707

710	709	709

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Daniela Ryšavá

806	801	801

807	802	802

In [58]:
evaluate(doc_sets, lemma_set, N, cosine_similarity_lemmalist)

Doc set by Josef Plch

106	101	101

107	103	103

108	104	104

109	104	104

110	105	105

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Vojtěch Škvařil

202	201	201

204	203	203

206	205	205

208	207	207

210	209	209

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Nikol Volková

306	301	301

307	303	302

308	303	303

309	303	304

310	301	305

Set precision: 0.40, recall: 1.00, F1: 0.57


Doc set by Lucie Kaplanová

402	401	401

404	403	403

406	405	405

408	407	407

410	409	409

Set precision: 1.00, recall: 1.00, F1: 1.00


Doc set by Matej Gallo

506	501	501

507	501	501

508	502	502

509	502	503

510	502	504

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Lukas Banic

606	605	601

607	604	602

609	604	604

610	605	605

613	604	611

Set precision: 0.40, recall: 1.00, F1: 0.57


Doc set by 422765

702	701	701

704	703	703

706	705	705

708	703	707

710	703	709

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Daniela Ryšavá

806	801	801

807	802	802

In [104]:
evaluate(doc_sets, lemma_set, N, tfidf_lemma)

Doc set by Josef Plch

106	101	101

107	103	103

108	104	104

109	102	104

110	105	105

Set precision: 0.80, recall: 1.00, F1: 0.89


Doc set by Vojtěch Škvařil

202	201	201

204	203	203

206	205	205

208	201	207

210	209	209

Set precision: 0.80, recall: 1.00, F1: 0.89


Doc set by Nikol Volková

306	304	301

307	304	302

308	303	303

309	304	304

310	305	305

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Lucie Kaplanová

402	401	401

404	403	403

406	405	405

408	407	407

410	403	409

Set precision: 0.80, recall: 1.00, F1: 0.89


Doc set by Matej Gallo

506	501	501

507	501	501

508	502	502

509	502	503

510	502	504

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Lukas Banic

606	611	601

607	611	602

609	611	604

610	605	605

613	611	611

Set precision: 0.40, recall: 1.00, F1: 0.57


Doc set by 422765

702	707	701

704	703	703

706	705	705

708	707	707

710	707	709

Set precision: 0.60, recall: 1.00, F1: 0.75


Doc set by Daniela Ryšavá

806	801	801

807	802	802